In [ ]:
import sqlite3
import pandas as pd
import numpy as np
import time

from collections import Counter

In [ ]:
con = sqlite3.connect('../../data/crawl.sqlite')

In [ ]:
sql = '''
SELECT r.* FROM recommendations r
LEFT JOIN searches s
  ON r.search_id=s.search_id
WHERE s.date = '2019-08-31'
    AND video_id NOT NULL
'''

recs = pd.read_sql_query(sql, con)
recs = recs.query("search_id == 34")

In [ ]:
np.random.seed(0)
def collapse_multiples(l, factor):
    '''
    Collapses `factor` duplicates of element in list into a single element.
    e.g. if l = [1,1,2,3,3,3,3] then collapse_multiples(l, 2) returns [1,2,3,3]
    
    INPUT:
        l: list
        factor: (int) collapse factor
    
    OUTPUT:
        collapsed list
    
    '''
    Counter(l)
    res = []
    for elem, n in Counter(l).items():
        if n % factor == 0:
            n /= factor
        res.extend([elem] * int(n))
    return res
    

def complete_tree(df, search_id, n_splits=4, const_depth=5):

    n_followed = []
    res = df.copy().filter(['video_id', 'recommendation', 'depth'])
    # create ids for each vertex
    res['vertex_id'] = (res
                        .groupby(['depth','video_id'])
                        .ngroup())
    # video depths and recs as dicts for fast lookup
    vid_depths = (df[['video_id', 'depth']]
                  .drop_duplicates()
                  .set_index('video_id')
                  .depth
                  .to_dict())
    vid_recs = (df[['video_id', 'recommendation']]
               .groupby('video_id')
               .agg(lambda x: list(x))
               .recommendation
               .to_dict())
    v_id = max(res.vertex_id.values)
    prev_recs = []
    for depth in df.depth.unique():
        # parent_ids == the video_ids for the depth we're currently at
        parent_ids = list((res
                     .query('depth == @depth')
                     .video_id
                     .unique()))
        # take the list difference (account for duplicates)
        truncd_ids = [i for i in prev_recs if not i in parent_ids or parent_ids.remove(i)]
        
        tic_v = time.time()
        for video_id in truncd_ids:
            v_id += 1
            if video_id is None or video_id not in vid_recs:
                continue
                
            # get the recommendations 
            recs = vid_recs[video_id]
            source_depth = vid_depths[video_id]
            if depth >= const_depth:
                n_followed.append(len(recs))
            
            # sample if we  are sampling, but our source recommendations were not sampled
            if depth >= const_depth and source_depth < const_depth:
                sample_inds = np.random.rand(len(recs)) < (1 / len(recs))
                recs = list(np.array(recs)[sample_inds])
            to_append_l = [[video_id, depth, v_id, rec] for rec in recs]
            to_append = pd.DataFrame(to_append_l,
                                    columns=['video_id','depth','vertex_id','recommendation'])
            res = res.append(to_append)
        toc_v = time.time()
        
        # update previous recs 
        prev_recs = list(res
            .query('depth == @depth')
            .recommendation
            .values)
        
        n_recs_depth = res.query("depth == @depth").shape[0]
        print("Number of recommendations at depth {}: {}".format(depth, n_recs_depth))
        if depth >= 7:
            print("Average sampled: {}".format(sum(n_followed) / len(n_followed)))
            return n_followed
            break

    res = res.assign(search_id=search_id).sort_values(['depth', 'video_id'])

    return res